# Familiarize with Dataset
Run the following cells to see dataset images with the points used as training data

In [1]:
import json
import numpy as np
import cv2
from scipy.spatial import distance as dist
import dlib
from imutils import face_utils


In [9]:
DATASET_SIZE = 10

with open('data_cleaned.json') as json_file:
    data = json.load(json_file)

# Convert python list to np array
keys_all = sorted(data)
keys_all = np.asarray(keys_all)
rand_indices = np.random.randint(len(data), size=DATASET_SIZE)
keys = keys_all[rand_indices]

Function that returns the required parameters of the solvePnP method. The paramaters *model_points* and *dist_coeffs* are the same for every example so they are declared globally.

In [3]:
def get_camera_parameters(size):
    # Approximate camera intrinsic parameters
    focal_length = size[1]
    center = (size[1]/2, size[0]/2)
    camera_matrix = np.array(
                             [[focal_length, 0, center[0]],
                             [0, focal_length, center[1]],
                             [0, 0, 1]], dtype = "double"
                             )
    dist_coeffs = np.zeros((4,1)) # Assuming no lens distortion
    
    return camera_matrix, dist_coeffs

In [4]:
def get_full_image_points(landmarks):
    image_points = np.zeros((68, 2))

    for i in range(68):
        image_points[i, :] = (landmarks[i]['x'], landmarks[i]['y'])
    
    return image_points

In [5]:
def get_full_model_points(filename='model_points.txt'):
    """Get all 68 3D model points from file"""
    raw_value = []
    with open(filename) as file:
        for line in file:
            raw_value.append(line)
    model_points = np.array(raw_value, dtype=np.float32)
    model_points = np.reshape(model_points, (3, -1)).T

    # Transform the model into a front view.
    model_points[:, 2] *= -1

    return model_points

In [6]:
def get_3D_reconstruction_points(name):
    filename = '../PRNet/TestImages/AFLW2000_results/' + name + '_kpt.txt'
    model_points = np.loadtxt(filename, dtype=np.float32)
    for i in range(len(model_points)):
        x = model_points[i][2]
        y = model_points[i][1]
        z = model_points[i][0]
        model_points[i] = [x, y, z]
    return model_points

In [7]:
def eye_aspect_ratio(eye):
    # compute the euclidean distances between the two sets of
    # vertical eye landmarks (x, y)-coordinates
    A = dist.euclidean([eye[1]['x'], eye[1]['y']], [eye[5]['x'], eye[5]['y']])
    B = dist.euclidean([eye[2]['x'], eye[2]['y']], [eye[4]['x'], eye[4]['y']])
 
    # compute the euclidean distance between the horizontal
    # eye landmark (x, y)-coordinates
    C = dist.euclidean([eye[0]['x'], eye[0]['y']], [eye[3]['x'], eye[3]['y']])
 
    # compute the eye aspect ratio
    ear = (A + B) / (2.0 * C)
 
    # return the eye aspect ratio
    return ear

Test the 3D model points for a test image

In [10]:
model_points = get_full_model_points()

for i in range(DATASET_SIZE):
    key = keys[i]
    im = cv2.imread('dataset/' + key)
    
    # Check if it is positive or negative example
    output = key.split('/')[1]
        
    ex = data[key]
    landmarks = ex['landmarks']
    iris_left = ex['iris_left']
    iris_right = ex['iris_right']
    
    # Change hardcoded size
    size = im.shape
    
    camera_matrix, dist_coeffs = get_camera_parameters(size)

    image_points = get_full_image_points(landmarks)

    # Solve the PnP problem with the parameters specified above
    (suc, rotation_vector, translation_vector) = cv2.solvePnP(
        model_points, image_points, camera_matrix, dist_coeffs, flags=cv2.SOLVEPNP_ITERATIVE
        )
    
    #Blink Detection
    leftEAR = eye_aspect_ratio(landmarks[36:42])
    rightEAR = eye_aspect_ratio(landmarks[42:48])
    ear = (leftEAR + rightEAR) / 2.0
    
    if(ear < 0.2):
        print('Blink in key #{}'.format(i))
    
    for point in image_points:
        cv2.circle(im, (int(point[0]), int(point[1])), 3, (0, 0, 255), -1)
    cv2.circle(im, (int(iris_left[0]), int(iris_left[1])), 3, (0, 0, 255), -1)
    cv2.circle(im, (int(iris_right[0]), int(iris_right[1])), 3, (0, 0, 255), -1)
    
    # Project the 3D point (0.55592, 6.5629, 300.0) onto the image plane.
    # We use this to draw a line sticking out of the nose
    (nose_end_point2D, jacobian) = cv2.projectPoints(
        np.array([(0.55592, 6.5629, 300.0)]), rotation_vector, translation_vector, camera_matrix, dist_coeffs
        )
    # Draw a line connecting the two points. This line must show
    # the direction out of the nose
    p1 = ( int(image_points[33][0]), int(image_points[33][1]) )
    p2 = ( int(nose_end_point2D[0][0][0]), int(nose_end_point2D[0][0][1]) )
    cv2.line(im, p1, p2, (255,0,0), 2)
    
    # Display image
    cv2.imshow(output, im)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

Blink in key #9
